### Problems: 
#### Longitudinal data of young children, we need to check if we take one entry each ? then we are only left with a few... 
#### subject ID as actually the match group (.xlsx)

In [51]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.io import mmread


In [52]:
#NOTE: There is a xlsx file in the folder that contains some info i try to integrate into the df´s


In [53]:
##TODO 
# what is disStage
### disease stage: paper specific
# what is Diff.From.Infant.Pos
### paper specific
# why is counts 119 and metadata 123 ?  
### See .xlsx file: 3 are excluded  

root_folder = '/var/datasets/single_papers/2023_Wimmers_CoVid_multi-omics/wimmers2023'

metadata = pd.read_csv(f"{root_folder}/bab_bulkRNA_metaData_220730.csv",sep=",")
metadata


,Subject.ID,PBMC.Collection.Date,Age at PBMC Collection (Months),Sex,Diff.From.Infant.Pos,disStage
0,CC0014,10/15/20,10.7,Male,-82.0,pre
1,CC0014,3/16/21,15.6,Male,70.0,conv
2,CC0016,12/15/20,12.8,Female,-58.0,pre
3,CC0016,7/1/21,19.3,Female,140.0,conv
4,CC0022,6/9/20,7.0,Female,-153.0,pre
...,...,...,...,...,...,...
118,PC183,10/6/20,31.0,Female,-135.0,pre
119,PC183,3/30/21,36.8,Female,40.0,conv
120,PC209,10/19/20,29.7,Male,-46.0,pre
121,PC209,6/16/21,37.6,Male,194.0,conv


In [54]:
rawdata = pd.read_csv(f"{root_folder}/raw_count_GRCh38_p22061_Florian_Analysis.txt",sep="\t", header=None).T
rawdata.columns = rawdata.iloc[0]
rawdata = rawdata.iloc[1:]
rawdata.head()
# ID vol sample: 'p22061-s001_barcode269931-subjectidCC0069-iridCR141-date20200605' 


/tmp/ipykernel_85840/3549991996.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  rawdata = pd.read_csv(f"{root_folder}/raw_count_GRCh38_p22061_Florian_Analysis.txt",sep="\t", header=None).T


,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Here i want to make it better readable what single entries mean (i split up that identifier string. .. )
columns_order = ['sample_id', 'barcode', 'subjectid', 'irid', 'date'] + list(rawdata.columns)
rawdata[['sample_id', 'barcode', 'subjectid', 'irid', 'date']] = rawdata['gene_id'].str.split('-', expand=True)
## add the first 4 chars of the barcode columns to the first one, as it is visible in the .xlsx file that it is supposed to be part of the sample_id
prefixes = rawdata['barcode'].str[:4]
rawdata['sample_id'] = rawdata['sample_id'] + '_' + prefixes
rawdata = rawdata[columns_order]
rawdata.head()

,sample_id,barcode,subjectid,irid,date,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061_s001,s001_barcode269931,subjectidCC0069,iridCR141,date20200605,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061_s002,s002_barcode270483,subjectidCC0022,iridCR074,date20200609,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061_s003,s003_barcode288558,subjectidCC0138,iridCR069,date20200715,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061_s004,s004_barcode322576,subjectidCC0256,iridCR169,date20200923,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061_s005,s005_barcode327849,subjectidCC0259,iridCR157,date20200928,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
prefixes = {
    'subjectid': 'subjectid',
    'irid': 'irid',
    'date': 'date'
}
for column, prefix in prefixes.items():
    rawdata[column] = rawdata[column].str.replace(f'^{prefix}', '', regex=True)
rawdata['barcode'] = rawdata['barcode'].str[12:] # buts the sxxx_barcode prefix
rawdata.head()

,sample_id,barcode,subjectid,irid,date,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061_s001,barcode269931,CC0069,CR141,20200605,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061_s002,barcode270483,CC0022,CR074,20200609,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061_s003,barcode288558,CC0138,CR069,20200715,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061_s004,barcode322576,CC0256,CR169,20200923,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061_s005,barcode327849,CC0259,CR157,20200928,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,...,0,0,0,0,0,0,0,0,0,0
